In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multinomial
from itertools import product

np.random.seed(0) # 控制种子实现得到相同的结论
# Given data
sku_costs = np.array([5, 6, 7, 8, 6, 2, 8, 5, 15, 7])
wtp_means = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
initial_price = np.array([6, 7.2, 8.4, 9.6, 7.2, 2.4, 9.6, 6, 18, 8.4])
purchases = np.array([2, 16, 7, 3, 10, 4, 6, 19, 11, 12])
possible_std_devs = np.array([0.5, 1, 2])
prior_prob = 1 / len(possible_std_devs)  # Each standard deviation is equally likely

# Number of customers
n_customers = 100
purchases = np.append(purchases, n_customers - purchases.sum())

# Function to compute the value distribution for a specific SKU
def compute_value_distribution_for_sku(sku_index, std):
    mean = wtp_means[sku_index]
    values = np.random.normal(loc=mean, scale=std, size=n_customers) - initial_price[sku_index]
    return values

# Generate all possible standard deviation combinations for all SKUs
std_dev_combinations = list(product(possible_std_devs, repeat=len(sku_costs)))

# Initialize the posterior array
likelihood = np.zeros((len(std_dev_combinations)))

In [2]:
def maxs(x):
    '''
    用于计算每个顾客最大价值的商品
    :param x: 输入的 NumPy 数组
    :return: 二进制数组，每行的最大值设为 1，其余为 0
    '''
    maxvalue = np.max(x, axis=1)
    result = np.zeros(x.shape)
    
    for i in range(x.shape[0]):
        if maxvalue[i] > 0: result[i, np.argmax(x[i])] = 1
    return result

In [3]:
temp = []
for idx, std_dev_comb in enumerate(std_dev_combinations):
    # Simulate value distributions for all SKUs based on the current std_dev_comb
    values = np.zeros((n_customers, len(sku_costs)))
    for i in range(len(sku_costs)):
        values[:, i] = compute_value_distribution_for_sku(i, std_dev_comb[i])
    # 模拟每个顾客每个商品的价值,每一行代表一个顾客，每一列代表一个商品，求每一行最大值
    simulated_purchases = maxs(values)
    simulated_purchases_prob = simulated_purchases.sum(axis = 0)
    totaltemp = np.sum(simulated_purchases_prob)
    simulated_purchases_prob = np.append(simulated_purchases_prob, 100 - totaltemp)
    temp.append(100 - totaltemp)
    simulated_purchases_prob = simulated_purchases_prob / n_customers
    log_likelihood = multinomial.logpmf(purchases, n_customers, simulated_purchases_prob)
    likelihood[idx]= np.exp(log_likelihood)

In [4]:
np.array(temp).sum(), len(std_dev_combinations)

(5847.0, 59049)

In [5]:
prior = np.full(len(possible_std_devs), 1 / len(possible_std_devs))

# Initialize the marginal likelihood array
marginal_likelihood = np.zeros((len(sku_costs)+1, len(possible_std_devs)))
marginal_likelihood_sum = np.zeros((len(sku_costs)+1))

# Calculate the marginal likelihood for each SKU and each assumption
posterior = pd.DataFrame(np.zeros((len(sku_costs)+1, len(possible_std_devs))))
for i in range(len(sku_costs) + 1):
    for j in range(len(possible_std_devs)):
        indices = [idx for idx, comb in enumerate(std_dev_combinations) if comb[i % len(comb)] == possible_std_devs[j]]
        marginal_likelihood[i, j] = np.sum(likelihood[indices])
    marginal_likelihood_sum[i] = np.sum(marginal_likelihood[i, :] * prior)
    posterior.iloc[i,:] = (marginal_likelihood[i,:] * prior_prob) / marginal_likelihood_sum[i]

In [6]:
posterior

,0,1,2
0,0.708938,0.287579,0.003482
1,0.000038,0.076618,0.923344
2,0.221079,0.703894,0.075027
3,0.745596,0.095790,0.158615
4,0.002355,0.799730,0.197915
5,0.279098,0.714269,0.006633
6,0.232710,0.763015,0.004275
7,0.000003,0.000870,0.999127
8,0.000400,0.789072,0.210528
9,0.074304,0.242442,0.683254
